In [ ]:
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np

In [ ]:
# Downloaded the data from this URL: https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page

# Load the data
df = pd.read_parquet('./data/yellow_tripdata_2023-01.parquet')

# Convert both timestamps to datetime
df.tpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime)
df.tpep_dropoff_datetime = pd.to_datetime(df.tpep_dropoff_datetime)

# Calculate duration in minutes
df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

In [ ]:
df.dtypes

**Q1. Downloading the data**

We'll use the same NYC taxi dataset, but instead of "Green Taxi Trip Records", we'll use "Yellow Taxi Trip Records".

Download the data for January and February 2023.

Read the data for January. How many columns are there?

- 19

In [ ]:
df.duration.describe()

**Q2. Computing duration**

Now let's compute the duration variable. It should contain the duration of a ride in minutes.

What's the standard deviation of the trips duration in January?

- 42.59

In [ ]:
# Store the original size
original_size = len(df)

# Filter for durations between 1 and 60 minutes (inclusive)
df = df[(df.duration >= 1) & (df.duration <= 60)]

# Calculate fraction of records left
fraction_left = len(df) / original_size

print(f"Original number of records: {original_size}")
print(f"Number of records after filtering: {len(df)}")
print(f"Fraction of records left: {fraction_left:.4f}")

**Q3. Dropping outliers**

Next, we need to check the distribution of the duration variable. There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

What fraction of the records left after you dropped the outliers?

- 98%

In [ ]:
# Keep only the required columns (PULocationID and DOLocationID)
df_features = df[['PULocationID', 'DOLocationID']]

# Convert dataframe to list of dictionaries with string conversion
dicts = df_features.astype(str).to_dict(orient='records')

# Create and fit a DictVectorizer
dv = DictVectorizer()
X = dv.fit_transform(dicts)

# Get the dimensionality (number of columns)
dimensionality = X.shape[1]

print(f"Feature matrix shape: {X.shape}")
print(f"Dimensionality (number of columns): {dimensionality}")

**Q4. One-hot encoding**

Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model.

Turn the dataframe into a list of dictionaries (remember to re-cast the ids to strings - otherwise it will label encode them)
Fit a dictionary vectorizer
Get a feature matrix from it
What's the dimensionality of this matrix (number of columns)?

- 515

In [ ]:
# Features (X) from previous step - the one-hot encoded pickup and dropoff locations
# Target variable (y) - duration
y = df['duration']

# Train a linear regression model
lr = LinearRegression()
lr.fit(X, y)

# Make predictions on the training data
y_pred = lr.predict(X)

# Calculate RMSE on the training data
rmse = np.sqrt(mean_squared_error(y, y_pred))

print(f"RMSE on training data: {rmse:.4f}")

**Q5. Training a model**

Now let's use the feature matrix from the previous step to train a model.

Train a plain linear regression model with default parameters, where duration is the response variable
Calculate the RMSE of the model on the training data
What's the RMSE on train?

- 7.64

In [ ]:
# Load the validation dataset (February 2023)
val_df = pd.read_parquet('./data/yellow_tripdata_2023-02.parquet')

# Apply the same preprocessing as we did for the training data
val_df.tpep_pickup_datetime = pd.to_datetime(val_df.tpep_pickup_datetime)
val_df.tpep_dropoff_datetime = pd.to_datetime(val_df.tpep_dropoff_datetime)

# Calculate duration in minutes
val_df['duration'] = val_df.tpep_dropoff_datetime - val_df.tpep_pickup_datetime
val_df.duration = val_df.duration.apply(lambda td: td.total_seconds() / 60)

# Filter for durations between 1 and 60 minutes (inclusive)
val_df = val_df[(val_df.duration >= 1) & (val_df.duration <= 60)]

# Prepare features - get only PULocationID and DOLocationID
val_features = val_df[['PULocationID', 'DOLocationID']]

# Convert to list of dictionaries with string conversion
val_dicts = val_features.astype(str).to_dict(orient='records')

# Transform validation data using the same DictVectorizer (don't refit)
X_val = dv.transform(val_dicts)

# Get validation target
y_val = val_df['duration']

# Make predictions using the trained model
y_val_pred = lr.predict(X_val)

# Calculate RMSE on validation data
val_rmse = np.sqrt(mean_squared_error(y_val, y_val_pred))

print(f"RMSE on validation data: {val_rmse:.4f}")

Q6. Evaluating the model

Now let's apply this model to the validation dataset (February 2023).

What's the RMSE on validation?

- 7.81